<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chap-19---Dyanmic-Attributes-and-Properties" data-toc-modified-id="Chap-19---Dyanmic-Attributes-and-Properties-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Chap 19 - Dyanmic Attributes and Properties</a></span><ul class="toc-item"><li><span><a href="#Jsonを読む" data-toc-modified-id="Jsonを読む-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Jsonを読む</a></span><ul class="toc-item"><li><span><a href="#ふつーにやる" data-toc-modified-id="ふつーにやる-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>ふつーにやる</a></span></li><li><span><a href="#feed[&quot;Schedule&quot;][&quot;speakers&quot;]--&gt;-feed.Schedule.speakers-とかがいい" data-toc-modified-id="feed[&quot;Schedule&quot;][&quot;speakers&quot;]-->-feed.Schedule.speakers-とかがいい-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>feed["Schedule"]["speakers"] -&gt; feed.Schedule.speakers とかがいい</a></span></li><li><span><a href="#What-if-Reserved-Keyword-is-attribute?" data-toc-modified-id="What-if-Reserved-Keyword-is-attribute?-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>What if Reserved Keyword is attribute?</a></span><ul class="toc-item"><li><span><a href="#invalid-syntax-にならなくなった" data-toc-modified-id="invalid-syntax-にならなくなった-1.1.3.1"><span class="toc-item-num">1.1.3.1&nbsp;&nbsp;</span>invalid syntax にならなくなった</a></span></li><li><span><a href="#頭に数字がついたキーもなんとかしたい" data-toc-modified-id="頭に数字がついたキーもなんとかしたい-1.1.3.2"><span class="toc-item-num">1.1.3.2&nbsp;&nbsp;</span>頭に数字がついたキーもなんとかしたい</a></span></li></ul></li><li><span><a href="#__new__-を使うとbuild-が消せる。" data-toc-modified-id="__new__-を使うとbuild-が消せる。-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>__new__ を使うとbuild が消せる。</a></span></li></ul></li><li><span><a href="#ORMっぽいことをする" data-toc-modified-id="ORMっぽいことをする-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>ORMっぽいことをする</a></span><ul class="toc-item"><li><span><a href="#event.venue.name-とかで引っ張ってこれないか?" data-toc-modified-id="event.venue.name-とかで引っ張ってこれないか?-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>event.venue.name とかで引っ張ってこれないか?</a></span><ul class="toc-item"><li><span><a href="#やってみる" data-toc-modified-id="やってみる-1.2.1.1"><span class="toc-item-num">1.2.1.1&nbsp;&nbsp;</span>やってみる</a></span></li></ul></li></ul></li><li><span><a href="#Using-a-property-for-attribute-validation" data-toc-modified-id="Using-a-property-for-attribute-validation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Using a property for attribute validation</a></span></li></ul></li></ul></div>

# Chap 19 - Dyanmic Attributes and Properties

## Jsonを読む

### ふつーにやる

In [35]:
import warnings
import os
import json


def load():
    with open("oscon_json/osconfeed.json") as fp:
        return json.load(fp)


feed = load()

for key, value in feed["Schedule"].items():
    print(f"{len(value)}:{key}")

1:conferences
484:events
357:speakers
53:venues


In [38]:
print(feed["Schedule"]["speakers"][40]["name"])
print(feed["Schedule"]["events"][40]["name"])

Tim Bray
There *Will* Be Bugs


### feed["Schedule"]["speakers"] -> feed.Schedule.speakers とかがいい

In [51]:
from collections import abc

class FrozenJSON:
    """Dict形式で、ValueがDictかList。
    DictでもListでもひたすら呼び出していって、
    最後にそれ以外の値がきたら、それを戻す。
    """
    
    def __init__(self,mapping):
        self.__data = dict(mapping)
        
    def __getattr__(self, name):
        
        # attributeとしてあるなら取る
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        
        # Attributeにないなら、keyをつかってValueを呼び出す
        else:
            return FrozenJSON.build(self.__data[name])

    @classmethod
    def build(cls, obj):
        
        # ValueがDictのとき
        if isinstance(obj, abc.Mapping):
            # もう一回FrozenJASONを作る
            return cls(obj)
        
        # ValueがListのとき
        elif isinstance(obj, abc.MutableSequence):
            # list elementごとにbuildをもう一回呼び出す
            return [cls.build(item) for item in obj]
        
        # それ以外はOBJを返す
        else:
            return obj

In [52]:
raw_feed = load()
feed = FrozenJSON(raw_feed)

In [53]:
print(feed.Schedule.speakers[40].name)
print(feed.Schedule.events[40].name)

Tim Bray
There *Will* Be Bugs


### What if Reserved Keyword is attribute?

class は予約語なのでエラーが出る

In [54]:
feed.class

SyntaxError: invalid syntax (<ipython-input-54-e42750f8d238>, line 1)

In [64]:
from collections import abc
import keyword

class FrozenJSON:
    """Dict形式で、ValueがDictかList。
    DictでもListでもひたすら呼び出していって、
    最後にそれ以外の値がきたら、それを戻す。
    """
    
    def __init__(self,mapping):
        self.__data = dict(mapping)
        
        # class だったら class_ にする。
        for key, value in mapping.items():
            # Return True if python keyword
            if keyword.iskeyword(key):
                key += '_'
                
            self.__data[key] = value
        
    def __getattr__(self, name):
        
        # attributeとしてあるなら取る
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        
        # Attributeにないなら、keyをつかってValueを呼び出す
        else:
            return FrozenJSON.build(self.__data[name])

    @classmethod
    def build(cls, obj):
        
        # ValueがDictのとき
        if isinstance(obj, abc.Mapping):
            # もう一回FrozenJASONを作る
            return cls(obj)
        
        # ValueがListのとき
        elif isinstance(obj, abc.MutableSequence):
            # list elementごとにbuildをもう一回呼び出す
            return [cls.build(item) for item in obj]
        
        # それ以外はOBJを返す
        else:
            return obj

#### invalid syntax にならなくなった

In [65]:
raw_feed = load()
feed = FrozenJSON(raw_feed)
feed.class_

KeyError: 'class_'

#### 頭に数字がついたキーもなんとかしたい

In [66]:
x = FrozenJSON({'2be': 'or not'})
x.2be

SyntaxError: invalid syntax (<ipython-input-66-5d4cd453fb3e>, line 2)

In [72]:
from collections import abc
import keyword


class FrozenJSON:
    """Dict形式で、ValueがDictかList。
    DictでもListでもひたすら呼び出していって、
    最後にそれ以外の値がきたら、それを戻す。
    """

    def __init__(self, mapping):
        self.__data = dict(mapping)

        # class だったら class_ にする。
        for key, value in mapping.items():
            # Return True if python keyword
            if keyword.iskeyword(key):
                key += "_"
            elif not key.isidentifier():
                key = f"_{key}"

            self.__data[key] = value

    def __getattr__(self, name):

        # attributeとしてあるなら取る
        if hasattr(self.__data, name):
            return getattr(self.__data, name)

        # Attributeにないなら、keyをつかってValueを呼び出す
        else:
            return FrozenJSON.build(self.__data[name])

    @classmethod
    def build(cls, obj):

        # ValueがDictのとき
        if isinstance(obj, abc.Mapping):
            # もう一回FrozenJASONを作る
            return cls(obj)

        # ValueがListのとき
        elif isinstance(obj, abc.MutableSequence):
            # list elementごとにbuildをもう一回呼び出す
            return [cls.build(item) for item in obj]

        # それ以外はOBJを返す
        else:
            return obj

In [74]:
x = FrozenJSON({'2be': 'or not'})
x._2be

'or not'

### \_\_new\_\_ を使うとbuild が消せる。

- \_\_init__ and \_\_new__
  - init are not allowed to return anything
  - new must return an instance
  
PYthonでは下のような感じでオブジェクトができてる、と思って良い

In [ ]:
def object_maker(the_class, some_arg):
    new_object = the_class.__new__(some_arg)
    if isinstance(new_object, the_class):
        the_class.__init__(new_object, some_arg)
    return new_object


x = Foo("bar")
x = object_maker(Foo, "bar")

In [82]:
from collections import abc
import keyword


class FrozenJSON:
    # build -> __new__
    def __new__(cls, arg):

        if isinstance(arg, abc.Mapping):
            # calling __new__ of object base class 
            return super().__new__(cls)

        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]

        else:
            return arg

    def __init__(self, mapping):
        self.__data = dict(mapping)
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += "_"
            elif not key.isidentifier():
                key = f"_{key}"

            self.__data[key] = value

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])

In [83]:
raw_feed = load()
feed = FrozenJSON(raw_feed)
print(feed.Schedule.speakers[40].name)

Tim Bray


## ORMっぽいことをする

簡単な実装。DB(ここではDict)にキーを入れてるだけ。

In [92]:
raw_feed = load()

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
def load_db(db):
    for collection, rec_list in raw_feed["Schedule"].items():
        record_type = collection[:-1]
        for record in rec_list:
            key = f"{record_type}, {record['serial']}"
            record["serial"] = key
            db[key] = Record(**record)
        
db = {}
load_db(db)
db

{'conference, 115': <__main__.Record at 0x7fc30a7ac0f0>,
 'event, 33451': <__main__.Record at 0x7fc2fbc57e80>,
 'event, 33457': <__main__.Record at 0x7fc2fbc57e10>,
 'event, 33463': <__main__.Record at 0x7fc2fba7a828>,
 'event, 33464': <__main__.Record at 0x7fc2fba7a860>,
 'event, 33476': <__main__.Record at 0x7fc2fba7a898>,
 'event, 33481': <__main__.Record at 0x7fc2fba7a8d0>,
 'event, 33485': <__main__.Record at 0x7fc2fba7a908>,
 'event, 33503': <__main__.Record at 0x7fc2fba7a940>,
 'event, 33520': <__main__.Record at 0x7fc2fba7a978>,
 'event, 33549': <__main__.Record at 0x7fc2fba7a9b0>,
 'event, 33557': <__main__.Record at 0x7fc2fba7a9e8>,
 'event, 33564': <__main__.Record at 0x7fc2fba7aa20>,
 'event, 33571': <__main__.Record at 0x7fc2fba7aa58>,
 'event, 33581': <__main__.Record at 0x7fc2fba7aa90>,
 'event, 33585': <__main__.Record at 0x7fc2fba7aac8>,
 'event, 33590': <__main__.Record at 0x7fc2fba7ab00>,
 'event, 33596': <__main__.Record at 0x7fc2fba7ab38>,
 'event, 33597': <__main_

### event.venue.name とかで引っ張ってこれないか?

In [135]:
import inspect


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __eq__(self, other):
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented


class MissingDatabaseError(RuntimeError):
    """DBがないときに警告"""


class Dbrecord(Record):
    __db = None
    
    # DB を設定
    @staticmethod
    def set_db(db):
        DbRecord.__db = db
    
    # DBオブジェクト(ここではDict)を返す
    @staticmethod
    def get_db(db):
        return DbRecord.__db
    
    # get_dbから指定されたキーを持ってくる
    @staticmethod
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]

        except TypeError:
            if db is None:
                msg = f"dtabase not set; call {cls.__name__}.set_db(my_db)"
                raise MissingDatabaseError(msg)
            else:
                raise

    def __repr__(self):
        if hasattr(self, "serial"):
            cls_name = self.__class__.__name__
            return f"<{cls_name} serial={self.serial}>"


class Event(DbRecord):
    """"モデルっぽいもの"""
    
    @property
    def venue(self):
        key = f"venue.{self.venue_serial}"
        return self.__class__.fetch(key)

    @property
    def speakers(self):
        # Speakerがなければ入れる
        if not hasattr(self, "_speaer_objs"):
            spkr_serials = self.__dict__["speakers"]
            fetch = self.__class__.fetch
            self._speaker_objs = [fetch(f"speaker.{key}") for key in spkr_serials]
        return self._speaker_objs

    def __repr__(self):
        if hasattr(self, "name"):
            cls_name = self.__class__.__name__
            return f"< {cls_name} {self.name}>"
        else:
            return super().__repr__()


def load_db(db):
    raw_data = load()
    for collection, rec_list in raw_data["Schedule"].items():
        record_type = collection[:-1]
        
        # クラス名を取得
        cls_name = record_type.capitalize()
        print(cls_name)
        
        # オブジェクト取得(ここでモデルであるEventを作る)
        cls = globals().get(cls_name, DbRecord)
        if inspect.isclass(cls) and issubclass(cls, DbRecord):
            factory = cls
        else:
            factory = DbRecord
        for record in rec_list:
            key = f"{record_type}.{record['serial']}"
            record["serial"] = key
            db[key] = factory(**record)

In [136]:
db = {}
load_db(db)

Conference
Event
Speaker
Venue


In [137]:
DbRecord.set_db(db)
event = DbRecord.fetch('event.33950')
event

< Event There *Will* Be Bugs>

In [138]:
event.venue

<DbRecord serial='venue.1449'>

In [139]:
event.venue.name

'Portland 251'

In [140]:
for spkr in event.speakers:
    print(f"{spkr.serial}: {spkr.name}")

speaker.3471: Anna Martelli Ravenscroft
speaker.5199: Alex Martelli


#### やってみる

In [38]:
my_db = {
    "Schedule": {
        "conferences": [{"serial": 115}],
        "events": [
            {
                "serial": 34505,
                "name": "Why Schools Don´t Use Open Source to Teach Programming",
                "event_type": "40-minute conference session",
                "time_start": "2014-07-23 11:30:00",
                "time_stop": "2014-07-23 12:10:00",
                "venue_serial": 1462,
                "description": "Aside from the fact that high school programming...",
                "website_url": "http://oscon.com/oscon2014/public/schedule/detail/34505",
                "speakers": [157509],
                "categories": ["Education"],
            },
            {
                "serial": 33451,
                "name": "Migrating to the Web Using Dart and Polymer - A Guide for Legacy OOP Developers",
                "event_type": "40-minute conference session",
                "time_start": "2014-07-23 17:00:00",
                "time_stop": "2014-07-23 17:40:00",
                "venue_serial": 1458,
                "description": "The web development platform is massive. With tons of libraries.",
                "website_url": "http://oscon.com/oscon2014/public/schedule/detail/33451",
                "speakers": [108840, 157509],
                "categories": ["Emerging Languages"],
            },
        ],
        "speakers": [
            {
                "serial": 157509,
                "name": "Robert Lefkowitz",
                "photo": None,
                "url": "http://sharewave.com/",
                "position": "CTO",
                "affiliation": "Sharewave",
                "twitter": "sharewaveteam",
                "bio": "Robert ´r0ml´ Lefkowitz is the CTO at Sharewave, a startup...",
            },
            {
                "serial": 108840,
                "name": "Ruth Suehle",
                "photo": "http://cdn.oreillystatic.com/en/assets/1/eventprovider/1/_@user_108840.jpg",
                "url": "http://www.opensource.com/",
                "position": "Marketing Manager",
                "affiliation": "Red Hat",
                "twitter": "suehle",
                "bio": "<p>Ruth Suehle is a community marketing manager</p>",
            },
        ],
        "venues": [
            {"serial": 1462, "name": "F151", "category": "Conference Venues"},
            {"serial": 1449, "name": "Portland 251", "category": "Conference Venues"},
        ],
    }
}

In [68]:
my_db["Schedule"]["conferences"]

[{'serial': 115}]

In [119]:
from collections import abc

class MyFrozenJSON():
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls(a_dict) for a_dict in arg]
        else:
            return arg
            
    
    def __init__(self, data):
        self.__data = data
        
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return MyFrozenJSON(self.__data[name])

In [121]:
MyFrozenJSON(my_db).Schedule.conferences[0].serial

115

## Using a property for attribute validation